## Imports

In [1]:
import copy
from functools import cache
import numpy as np

In [2]:
day = "21"

In [3]:
with open(f"input_j{day}.txt") as f:
# with open("test.txt") as f:
    garden = f.readlines()
    garden = list(map(lambda x: x.replace("\n", ""), garden))
    garden = list(map(lambda x: [case for case in x], garden))

n, m = len(garden), len(garden[0])
print(n, m)

131 131


## Part 1

### Départ

In [4]:
def find_start():
    for i in range(n):
        for j in range(m):
            if garden[i][j] == "S":
                return (i, j)

In [5]:
i_start, j_start = find_start()
i_start, j_start

(65, 65)

### Voisins & distances

In [6]:
voisins_1 = copy.deepcopy(garden)
for i in range(n):
    for j in range(m):
        voisins_1[i][j] = set()

for i in range(n):
    for j in range(m):
        if i>0 and garden[i-1][j] in ["S", "."]:
            voisins_1[i][j].add((i-1, j))
        if i<n-1 and garden[i+1][j] in ["S", "."]:
            voisins_1[i][j].add((i+1, j))
        if j>0 and garden[i][j-1] in ["S", "."]:
            voisins_1[i][j].add((i, j-1))
        if j<m-1 and garden[i][j+1] in ["S", "."]:
            voisins_1[i][j].add((i, j+1))

In [7]:
distances_1 = copy.deepcopy(garden)
for i in range(n):
    for j in range(m):
        distances_1[i][j] = float('inf')
distances_1[i_start][j_start] = 0

### Dijkstra

In [8]:
def find_min(Q, distances):
    mini = float('inf')
    for node in Q:
        if distances[node[0]][node[1]] <= mini:
            mini = distances[node[0]][node[1]]
            sommet = node
    return sommet

In [9]:
def update_distances(node_1, node_2, distances):
    if distances[node_2[0]][node_2[1]] > distances[node_1[0]][node_1[1]] + 1:
        distances[node_2[0]][node_2[1]] = distances[node_1[0]][node_1[1]] + 1
    return distances

In [10]:
def dijkstra(i_start, j_start, voisins=voisins_1, distances=distances_1, nb_pas_allowed = 64):
    Q = set([(i_start, j_start)])
    not_Q = set()
    while len(Q) > 0:
        # if len(Q) % 2000 == 0:
            # print(len(Q))
        node_1 = find_min(Q, distances)
        if distances[node_1[0]][node_1[1]] > nb_pas_allowed:
           break
        Q.remove(node_1)
        not_Q.add(node_1)
        for node_2 in voisins[node_1[0]][node_1[1]]:
            distances = update_distances(node_1, node_2, distances)
            if node_2 not in not_Q:
                Q.add(node_2)
        # print(Q)
    return distances

### Application

In [11]:
nb_pas_allowed_1 = 64
# nb_pas_allowed_1 = 6
parite_1 = nb_pas_allowed_1 % 2

In [12]:
distances_1 = dijkstra(i_start, j_start, voisins_1, distances_1, nb_pas_allowed_1)

In [13]:
compteur_1 = 0
for i in range(n):
    for j in range(m):
        if distances_1[i][j] <= nb_pas_allowed_1 and distances_1[i][j] % 2 == parite_1:
            compteur_1 += 1

compteur_1

3632

## Part 2

In [14]:
nb_pas_allowed_2 = 26501365
# nb_pas_allowed_2 = 6
nb_pas_allowed_2 % n  # This is the starting position

65

In [15]:
def interpolation(x, nb_pas_allowed):

    parite = nb_pas_allowed % 2

    new_garden = copy.deepcopy(garden)
    
    for i in range(2*x):
        new_garden.extend(garden)
    new_garden = np.transpose(new_garden).tolist()
    copy_new_garden = copy.deepcopy(new_garden)
    for j in range(2*x):
        new_garden.extend(copy_new_garden)
    new_garden = np.transpose(new_garden).tolist()
    n_i = len(new_garden)
    m_i = len(new_garden[0])
    print(n_i, m_i)

    voisins = copy.deepcopy(new_garden)
    for i in range(n_i):
        for j in range(m_i):
            voisins[i][j] = set()
    for i in range(n_i):
        for j in range(m_i):
            if i>0 and new_garden[i-1][j] in ["S", "."]:
                voisins[i][j].add((i-1, j))
            if i<n_i-1 and new_garden[i+1][j] in ["S", "."]:
                voisins[i][j].add((i+1, j))
            if j>0 and new_garden[i][j-1] in ["S", "."]:
                voisins[i][j].add((i, j-1))
            if j<m_i-1 and new_garden[i][j+1] in ["S", "."]:
                voisins[i][j].add((i, j+1))

    new_i_start, new_j_start = i_start + x * n, j_start + x * m
    distances = copy.deepcopy(new_garden)
    for i in range(n_i):
        for j in range(m_i):
            distances[i][j] = float('inf')
    distances[new_i_start][new_j_start] = 0

    # print(i_start, j_start, len(voisins), len(distances), nb_pas_allowed)
    distances = dijkstra(new_i_start, new_j_start, voisins, distances, nb_pas_allowed)
    #for row in new_garden:
    #    print(row)
    #for row in voisins:
    #    print(row)

    compteur = 0
    for i in range(n_i):
        for j in range(m_i):
            if distances[i][j] <= nb_pas_allowed and distances[i][j] % 2 == parite:
                compteur += 1

    return compteur

In [16]:
interpolation(0, nb_pas_allowed_1)

131 131


3632

In [17]:
for i in range(4):
    print("Résultat : ", interpolation(i, (nb_pas_allowed_2 % n) + i * n))

131 131
Résultat :  3701
393 393
Résultat :  33108
655 655
Résultat :  91853
917 917
Résultat :  179936


Let's see if a quadratic interpolation fits:

https://www.wolframalpha.com/input?i=quadratic+fit+calculator&assumption=%7B%22F%22%2C+%22QuadraticFitCalculator%22%2C+%22data3x%22%7D+-%3E%22%7B0%2C+1%2C+2%7D%22&assumption=%7B%22F%22%2C+%22QuadraticFitCalculator%22%2C+%22data3y%22%7D+-%3E%22%7B3701%2C+33108%2C+91853%7D%22

In [18]:
def interpolation_polynomiale(i):
    return (14669 * i**2 + 14738 * i + 3701)

In [19]:
for i in range(4):
    print(interpolation_polynomiale(i))

3701
33108
91853
179936


In [20]:
k = nb_pas_allowed_2 // n
interpolation_polynomiale(k)

600336060511101